In [1]:
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils.vis_utils import plot_model
from keras.applications.mobilenet_v2 import decode_predictions
from keras.layers.normalization import BatchNormalization

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os

import tensorflow
from time import time

import pandas

import pickle as pk

from keras.wrappers.scikit_learn import KerasClassifier

import numpy as np

seed = 7

Using TensorFlow backend.


## Hyperparameter runing on mobilenet with Adagrad.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11223541811075911503
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11280557671
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17597718168750094962
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


In [3]:
dir(keras.applications.mobilenet_v2)

['MobileNetV2',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'absolute_import',
 'decode_predictions',
 'division',
 'keras_modules_injection',
 'mobilenet_v2',
 'preprocess_input',
 'print_function']

In [4]:
np.random.seed(seed)

In [5]:
data_dir = '/data/oxford102/train'
train_data_dir = "/data/oxford102/train/"
validation_data_dir = "/data/oxford102/train/"
img_width, img_height = 256, 256
batch_size = 128
epochs = 100
nr_categories = 102
nb_train_samples = 4604
nb_validation_samples = 1094 

In [6]:
def get_image_generator(input_processor, img_aug=False):
    if not img_aug:
        train_val_datagen = ImageDataGenerator(preprocessing_function=input_processor, 
                                           validation_split=0.2)
    else: 
        train_val_datagen = ImageDataGenerator(
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest',
            preprocessing_function=input_processor,
            validation_split=0.2)
        
    return train_val_datagen

In [7]:
def get_generators(batch_size, image_size, input_processor, img_aug=False):

    img_width, img_height = image_size
    
    train_val_datagen = get_image_generator(input_processor, img_aug)

    train_generator = train_val_datagen.flow_from_directory(
            train_data_dir,  # this is the target directory
            target_size=(img_width, img_height),  # all images will be resized to 250x250
            batch_size=batch_size,
            subset="training",
            class_mode='categorical')

    validation_generator = train_val_datagen.flow_from_directory(
            train_data_dir,  # this is the target directory
            target_size=(img_width, img_height),  # all images will be resized to 250x250
            subset="validation",
            batch_size=batch_size,
            class_mode='categorical')
    
    return train_generator, validation_generator


In [8]:
#from keras import regularizers

def get_model(network_name="inception_resnet_v2", image_size=(256, 256), verbose=False):
    k.set_learning_phase(0)

    img_width, img_height = image_size
    if network_name == "vgg16":
        base_model = keras.applications.vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
        input_processor = applications.vgg16.preprocess_input
    elif network_name == "vgg19":
        base_model = keras.applications.vgg19.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
        input_processor = applications.vgg19.preprocess_input
    elif network_name == "inception_resnet_v2":
        base_model = keras.applications.inception_resnet_v2.InceptionResNetV2(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
        input_processor = applications.inception_resnet_v2.preprocess_input
    elif network_name == "mobilenet_v2":
        base_model = keras.applications.mobilenet_v2.MobileNetV2(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
        input_processor = applications.mobilenet_v2.preprocess_input
    else:
        raise Exception("check your network name")

    for layer in base_model.layers[:]:
        layer.trainable = False

        #Adding custom Layers 
    k.set_learning_phase(1)
    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation="relu", 
              #kernel_regularizer=regularizers.l2(0.01),
             #       activity_regularizer=regularizers.l1(0.001)
             )(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x, training=True)
    #x = Dense(102, activation="relu")(x)
    predictions = Dense(nr_categories, activation="softmax")(x)

    _model = Model(input = base_model.input, output = predictions)
    if verbose:
        _model.summary()
    return _model, input_processor

In [ ]:
def train_model(params, _model, generators):
    model_name = params["network_name"]
    num_train_img = 4604
    num_val_img = 1094

    np.random.seed(seed)
    log_time = time()
    params['log_time'] = log_time
    batch_size = params.get("batch_size")

    train_generator, validation_generator = generators
    _model.compile(loss = "categorical_crossentropy", optimizer = params["optimizer"], metrics=["accuracy"])


    base = '/data/oxford102/experiments'
    path = os.path.join(base, str(log_time))
    checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
    tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
    csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

    try:
        if not os.path.exists(path):
            os.makedirs(path)
        history_callback = _model.fit_generator(
                train_generator,
                steps_per_epoch=num_train_img // params["batch_size"] // 2,
                epochs=params["epochs"],
                validation_data=validation_generator,
                validation_steps=num_val_img // params["batch_size"],
                callbacks = [checkpoint, early, tensorboard, csv_logger])
    except Exception as e:
        raise(e)
    finally:
        params.pop("optimizer")
        pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
        _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
        print(params)
        params

In [9]:
# Hyperparameter optimization

In [10]:
optimizers

<module 'keras.optimizers' from '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/optimizers.py'>

In [ ]:
optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [ ]:
optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)

In [ ]:
optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

In [ ]:
optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)

In [ ]:
optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)

In [ ]:
optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [ ]:
optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)

In [ ]:
optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

In [10]:
#import talos as ta

# p = {'lr': (0.001, 0.01, 0.1),
#      'batch_size': (128),
#      'epochs': [150],
#      'dropout': (0, 0.5, 5),
#      'weight_regulizer':[None],
#      'emb_output_dims': [None],
#      'shape':['brick','long_funnel'],
#      'optimizer': [Adam, Nadam, RMSprop],
#      'losses': [logcosh, binary_crossentropy],
#      'activation':[relu, elu],
#      'last_activation': [sigmoid]}

In [9]:
# def train_model(params, _model, generators):
#     model_name = params["network_name"]
#     num_train_img = 4604
#     num_val_img = 1094

#     np.random.seed(seed)
#     log_time = time()
#     params['log_time'] = log_time
#     batch_size = params.get("batch_size")

#     train_generator, validation_generator = generators
#     _model.compile(loss = "categorical_crossentropy", optimizer = params["optimizer"], metrics=["accuracy"])


#     base = '/data/oxford102/experiments'
#     path = os.path.join(base, str(log_time))
#     checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
#     early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
#     tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
#     csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

#     try:
#         if not os.path.exists(path):
#             os.makedirs(path)
#         history_callback = _model.fit_generator(
#                 train_generator,
#                 steps_per_epoch=num_train_img // params["batch_size"] // 2,
#                 epochs=params["epochs"],
#                 validation_data=validation_generator,
#                 validation_steps=num_val_img // params["batch_size"],
#                 callbacks = [checkpoint, early, tensorboard, csv_logger])
#     except Exception as e:
#         raise(e)
#     finally:
#         params.pop("optimizer")
#         pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
#         _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
#         print(params)
#         params

# MobileNet + adagrad

In [10]:
import itertools
import numpy

In [11]:
lr = [0.1, 0.01, 0.001]
rho = [0.5, 0.75, 0.9]
epsilon = [None, 1e-06, 1e-04, 1e-02]
decay = [0.001, 0.01, 0.1, 0.2]


In [12]:
# from keras import backend as K
# K.epsilon()
# >>> 1e-7

In [13]:
grid = list(itertools.product(lr, rho, epsilon, decay))

In [14]:
grid

[(0.1, 0.5, None, 0.001),
 (0.1, 0.5, None, 0.01),
 (0.1, 0.5, None, 0.1),
 (0.1, 0.5, None, 0.2),
 (0.1, 0.5, 1e-06, 0.001),
 (0.1, 0.5, 1e-06, 0.01),
 (0.1, 0.5, 1e-06, 0.1),
 (0.1, 0.5, 1e-06, 0.2),
 (0.1, 0.5, 0.0001, 0.001),
 (0.1, 0.5, 0.0001, 0.01),
 (0.1, 0.5, 0.0001, 0.1),
 (0.1, 0.5, 0.0001, 0.2),
 (0.1, 0.5, 0.01, 0.001),
 (0.1, 0.5, 0.01, 0.01),
 (0.1, 0.5, 0.01, 0.1),
 (0.1, 0.5, 0.01, 0.2),
 (0.1, 0.75, None, 0.001),
 (0.1, 0.75, None, 0.01),
 (0.1, 0.75, None, 0.1),
 (0.1, 0.75, None, 0.2),
 (0.1, 0.75, 1e-06, 0.001),
 (0.1, 0.75, 1e-06, 0.01),
 (0.1, 0.75, 1e-06, 0.1),
 (0.1, 0.75, 1e-06, 0.2),
 (0.1, 0.75, 0.0001, 0.001),
 (0.1, 0.75, 0.0001, 0.01),
 (0.1, 0.75, 0.0001, 0.1),
 (0.1, 0.75, 0.0001, 0.2),
 (0.1, 0.75, 0.01, 0.001),
 (0.1, 0.75, 0.01, 0.01),
 (0.1, 0.75, 0.01, 0.1),
 (0.1, 0.75, 0.01, 0.2),
 (0.1, 0.9, None, 0.001),
 (0.1, 0.9, None, 0.01),
 (0.1, 0.9, None, 0.1),
 (0.1, 0.9, None, 0.2),
 (0.1, 0.9, 1e-06, 0.001),
 (0.1, 0.9, 1e-06, 0.01),
 (0.1, 0.9, 1e-0

In [7]:
choices = numpy.random.choice(range(len(grid)), int(len(grid)*0.2), replace=False)

In [8]:
# choices
# >>> array([ 11, 122,  37,  32,  70, 113,  77, 116, 134,  74, 143,  66,  41,
#         81, 102,  97,  62,  49,  51,  80,  36,  91,  65,  98,  76, 135,
#         40,  45])

In [11]:
choices = np.array([ 11, 122,  37,  32,  70, 113,  77, 116, 134,  74, 143,  66,  41,
        81, 102,  97,  62,  49,  51,  80,  36,  91,  65,  98,  76, 135,
        40,  45])

In [ ]:
choices = np.array([11, 32, 36, 37, 40, 41, 45, 49, 51, 62, 65, 66, 70, 74, 76, 77, 80, 81, 91, 97, 98, 102, 113, 116, 122, 134, 135, 143])

In [27]:
params = {'network_name': "mobilenet_v2",
         'image_aug': False,
         'optimizer_name': "adagrad", 
         'optimizer': optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
         'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (224, 224),
         'log_time': None}

In [26]:
_optimizer = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 20s 1s/step - loss: 2.5101 - acc: 0.4504 - val_loss: 1.5273 - val_acc: 0.6328

Epoch 00001: val_acc improved from -inf to 0.63281, saving model to /data/oxford102/experiments/1546265924.8866432/mobilenet_v2_1546265924.8866432.h5
Epoch 2/100
17/17 [==============================] - 9s 541ms/step - loss: 1.2092 - acc: 0.7073 - val_loss: 1.0390 - val_acc: 0.7226

Epoch 00002: val_acc improved from 0.63281 to 0.72257, saving model to /data/oxford102/experiments/1546265924.8866432/mobilenet_v2_1546265924.8866432.h5
Epoch 3/100
17/17 [==============================] - 13s 767ms/step - loss: 0.3889 - acc: 0.9183 - val_loss: 0.7759 - val_acc: 0.8033

Epoch 00003: val_acc improved from 0.72257 to 0.80331, saving model to /data/oxford102/experiments/1546265924.8866432/mobilenet_v2_1546265924.8866432.h5
Epoch 4/100
17/17 [==============================] - 14

In [27]:
params = {'network_name': "vgg16",
         'image_aug': False,
         'optimizer_name': "adagrad", 
         'optimizer': optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
         'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (224, 224),
         'log_time': None}

In [28]:
_optimizer = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

58892288/58889256 [==============================] - 1s 0us/step


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 38s 2s/step - loss: 2.3898 - acc: 0.4839 - val_loss: 1.4045 - val_acc: 0.6621

Epoch 00001: val_acc improved from -inf to 0.66211, saving model to /data/oxford102/experiments/1546266222.6804273/vgg16_1546266222.6804273.h5
Epoch 2/100
17/17 [==============================] - 33s 2s/step - loss: 1.0890 - acc: 0.7367 - val_loss: 0.9688 - val_acc: 0.7578

Epoch 00002: val_acc improved from 0.66211 to 0.75776, saving model to /data/oxford102/experiments/1546266222.6804273/vgg16_1546266222.6804273.h5
Epoch 3/100
17/17 [==============================] - 36s 2s/step - loss: 0.2759 - acc: 0.9538 - val_loss: 0.8252 - val_acc: 0.7992

Epoch 00003: val_acc improved from 0.75776 to 0.79917, saving model to /data/oxford102/experiments/1546266222.6804273/vgg16_1546266222.6804273.h5
Epoch 4/100
17/17 [==============================] - 28s 2s/step - loss: 0.1377 -

In [ ]:
params = {'network_name': "vgg16",
         'image_aug': False,
         'optimizer_name': "adagrad", 
         'optimizer': optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
         'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (224, 224),
         'log_time': None}

In [ ]:
_optimizer = optimizers.Adagrad(lr=0.001, epsilon=None, decay=0.0)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

In [29]:
params = {'network_name': "vgg19",
         'image_aug': False,
         'optimizer_name': "adagrad", 
         'optimizer': optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
         'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (224, 224),
         'log_time': None}

In [30]:
_optimizer = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 34s 2s/step - loss: 2.4160 - acc: 0.4738 - val_loss: 1.4415 - val_acc: 0.6650

Epoch 00001: val_acc improved from -inf to 0.66504, saving model to /data/oxford102/experiments/1546266538.4235597/vgg19_1546266538.4235597.h5
Epoch 2/100
17/17 [==============================] - 32s 2s/step - loss: 1.1344 - acc: 0.7215 - val_loss: 1.0210 - val_acc: 0.7557

Epoch 00002: val_acc improved from 0.66504 to 0.75569, saving model to /data/oxford102/experiments/1546266538.4235597/vgg19_1546266538.4235597.h5
Epoch 3/100
17/17 [==============================] - 32s 2s/step - loss: 0.2891 - acc: 0.9509 - val_loss: 0.8713 - val_acc: 0.7785

Epoch 00003: val_acc improved from 0.75569 to 0.77847, saving model to /data/oxford102/experiments/1546266538.4235597/vgg19_1546266538.4235597.h5
Epoch 4/100
17/17 [==============================] - 32s 2s/step - loss: 0.1565 -

In [ ]:
params = {'network_name': "vgg19",
         'image_aug': False,
         'optimizer_name': "adagrad", 
         'optimizer': optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
         'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (224, 224),
         'log_time': None}

In [ ]:
_optimizer = optimizers.Adagrad(lr=0.001, epsilon=None, decay=0.0)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

In [31]:
params = {'network_name': "inception_resnet_v2",
         'image_aug': False,
         'optimizer_name': "adagrad", 
         'optimizer': optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
         'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (224, 224),
         'log_time': None}

In [32]:
_optimizer = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 43s 3s/step - loss: 3.3648 - acc: 0.2564 - val_loss: 2.5377 - val_acc: 0.3652

Epoch 00001: val_acc improved from -inf to 0.36523, saving model to /data/oxford102/experiments/1546266852.4233403/inception_resnet_v2_1546266852.4233403.h5
Epoch 2/100
17/17 [==============================] - 36s 2s/step - loss: 2.3679 - acc: 0.4205 - val_loss: 2.0556 - val_acc: 0.4731

Epoch 00002: val_acc improved from 0.36523 to 0.47308, saving model to /data/oxford102/experiments/1546266852.4233403/inception_resnet_v2_1546266852.4233403.h5
Epoch 3/100
17/17 [==============================] - 37s 2s/step - loss: 1.5736 - acc: 0.6048 - val_loss: 1.7424 - val_acc: 0.5538

Epoch 00003: val_acc improved from 0.47308 to 0.55383, saving model to /data/oxford102/experiments/1546266852.4233403/inception_resnet_v2_1546266852.4233403.h5
Epoch 4/100
17/17 [====================

In [ ]:
params = {'network_name': "inception_resnet_v2",
         'image_aug': False,
         'optimizer_name': "adagrad", 
         'optimizer': optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
         'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (224, 224),
         'log_time': None}

In [ ]:
_optimizer = optimizers.Adagrad(lr=0.001, epsilon=None, decay=0.0)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

# Adadelta

In [10]:
params = {'network_name': "mobilenet_v2",
         'image_aug': False,
         'optimizer_name': "adagdelta", 
         'optimizer': optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
         'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (224, 224),
         'log_time': None}

In [34]:
lr, epsilon, decay = (0.01, None, 0.0)
_optimizer = optimizers.Adadelta(lr=0.01, rho=0.95, epsilon=None, decay=0.0)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 18s 1s/step - loss: 4.9817 - acc: 0.0395 - val_loss: 4.4178 - val_acc: 0.0830

Epoch 00001: val_acc improved from -inf to 0.08301, saving model to /data/oxford102/experiments/1546267945.754025/mobilenet_v2_1546267945.754025.h5
Epoch 2/100
17/17 [==============================] - 9s 543ms/step - loss: 4.0254 - acc: 0.1420 - val_loss: 3.6703 - val_acc: 0.2019

Epoch 00002: val_acc improved from 0.08301 to 0.20186, saving model to /data/oxford102/experiments/1546267945.754025/mobilenet_v2_1546267945.754025.h5
Epoch 3/100
17/17 [==============================] - 12s 678ms/step - loss: 2.9283 - acc: 0.3488 - val_loss: 3.2601 - val_acc: 0.2899

Epoch 00003: val_acc improved from 0.20186 to 0.28986, saving model to /data/oxford102/experiments/1546267945.754025/mobilenet_v2_1546267945.754025.h5
Epoch 4/100
17/17 [==============================] - 13s 787m

In [35]:
params['network_name'] = "inception_resnet_v2"
_optimizer = optimizers.Adadelta(lr=0.01, rho=0.95, epsilon=None, decay=0.0)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 42s 2s/step - loss: 4.8742 - acc: 0.0423 - val_loss: 4.3529 - val_acc: 0.0850

Epoch 00001: val_acc improved from -inf to 0.08496, saving model to /data/oxford102/experiments/1546268556.0703526/inception_resnet_v2_1546268556.0703526.h5
Epoch 2/100
17/17 [==============================] - 35s 2s/step - loss: 4.0917 - acc: 0.1314 - val_loss: 3.6941 - val_acc: 0.1656

Epoch 00002: val_acc improved from 0.08496 to 0.16563, saving model to /data/oxford102/experiments/1546268556.0703526/inception_resnet_v2_1546268556.0703526.h5
Epoch 3/100
17/17 [==============================] - 35s 2s/step - loss: 3.3010 - acc: 0.2568 - val_loss: 3.2608 - val_acc: 0.2578

Epoch 00003: val_acc improved from 0.16563 to 0.25776, saving model to /data/oxford102/experiments/1546268556.0703526/inception_resnet_v2_1546268556.0703526.h5
Epoch 4/100
17/17 [====================

In [11]:
params['network_name'] = "vgg16"
_optimizer = optimizers.Adadelta(lr=0.01, rho=0.95, epsilon=None, decay=0.0)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 38s 2s/step - loss: 5.3186 - acc: 0.0124 - val_loss: 5.2461 - val_acc: 0.0107

Epoch 00001: val_acc improved from -inf to 0.01074, saving model to /data/oxford102/experiments/1546270611.0157263/vgg16_1546270611.0157263.h5
Epoch 2/100
17/17 [==============================] - 33s 2s/step - loss: 5.0722 - acc: 0.0267 - val_loss: 5.0202 - val_acc: 0.0197

Epoch 00002: val_acc improved from 0.01074 to 0.01967, saving model to /data/oxford102/experiments/1546270611.0157263/vgg16_1546270611.0157263.h5
Epoch 3/100
17/17 [==============================] - 36s 2s/step - loss: 4.6364 - acc: 0.0543 - val_loss: 4.6999 - val_acc: 0.0424

Epoch 00003: val_acc improved from 0.01967 to 0.04244, saving model to /data/oxford102/experiments/1546270611.0157263/vgg16_1546270611.0157263.h5
Epoch 4/100
17/17 [==============================] - 28s 2s/step - loss: 4.3913 -

Epoch 32/100
17/17 [==============================] - 28s 2s/step - loss: 0.9988 - acc: 0.8532 - val_loss: 2.3451 - val_acc: 0.4886

Epoch 00032: val_acc did not improve from 0.51346
Epoch 33/100
17/17 [==============================] - 28s 2s/step - loss: 0.9519 - acc: 0.8695 - val_loss: 2.1127 - val_acc: 0.5373

Epoch 00033: val_acc improved from 0.51346 to 0.53727, saving model to /data/oxford102/experiments/1546270611.0157263/vgg16_1546270611.0157263.h5
Epoch 34/100
17/17 [==============================] - 28s 2s/step - loss: 0.9438 - acc: 0.8716 - val_loss: 2.1885 - val_acc: 0.5104

Epoch 00034: val_acc did not improve from 0.53727
Epoch 35/100
17/17 [==============================] - 28s 2s/step - loss: 0.8884 - acc: 0.8906 - val_loss: 2.1396 - val_acc: 0.5248

Epoch 00035: val_acc did not improve from 0.53727
Epoch 36/100
17/17 [==============================] - 28s 2s/step - loss: 0.8758 - acc: 0.8845 - val_loss: 2.0965 - val_acc: 0.5383

Epoch 00036: val_acc improved from 0.53

In [12]:
params['network_name'] = "vgg19"
_optimizer = optimizers.Adadelta(lr=0.01, rho=0.95, epsilon=None, decay=0.0)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 34s 2s/step - loss: 5.3990 - acc: 0.0119 - val_loss: 5.3062 - val_acc: 0.0176

Epoch 00001: val_acc improved from -inf to 0.01758, saving model to /data/oxford102/experiments/1546272025.2986946/vgg19_1546272025.2986946.h5
Epoch 2/100
17/17 [==============================] - 32s 2s/step - loss: 5.1076 - acc: 0.0221 - val_loss: 4.9282 - val_acc: 0.0404

Epoch 00002: val_acc improved from 0.01758 to 0.04037, saving model to /data/oxford102/experiments/1546272025.2986946/vgg19_1546272025.2986946.h5
Epoch 3/100
17/17 [==============================] - 32s 2s/step - loss: 4.6636 - acc: 0.0497 - val_loss: 4.7241 - val_acc: 0.0487

Epoch 00003: val_acc improved from 0.04037 to 0.04865, saving model to /data/oxford102/experiments/1546272025.2986946/vgg19_1546272025.2986946.h5
Epoch 4/100
17/17 [==============================] - 32s 2s/step - loss: 4.3632 -

Epoch 34/100
17/17 [==============================] - 32s 2s/step - loss: 0.9839 - acc: 0.8472 - val_loss: 2.1930 - val_acc: 0.5083

Epoch 00034: val_acc did not improve from 0.52381
Epoch 35/100
17/17 [==============================] - 32s 2s/step - loss: 0.8987 - acc: 0.8847 - val_loss: 2.1343 - val_acc: 0.5300

Epoch 00035: val_acc improved from 0.52381 to 0.53002, saving model to /data/oxford102/experiments/1546272025.2986946/vgg19_1546272025.2986946.h5
Epoch 36/100
17/17 [==============================] - 32s 2s/step - loss: 0.8865 - acc: 0.8684 - val_loss: 2.1323 - val_acc: 0.5352

Epoch 00036: val_acc improved from 0.53002 to 0.53520, saving model to /data/oxford102/experiments/1546272025.2986946/vgg19_1546272025.2986946.h5
Epoch 37/100
17/17 [==============================] - 32s 2s/step - loss: 0.8302 - acc: 0.9017 - val_loss: 2.0710 - val_acc: 0.5488

Epoch 00037: val_acc improved from 0.53520 to 0.54883, saving model to /data/oxford102/experiments/1546272025.2986946/vgg19_15

# Adam

In [12]:
params = {'network_name': "mobilenet_v2",
         'image_aug': False,
         'optimizer_name': "adam", 
         'optimizer': optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
         'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (224, 224),
         'log_time': None}

In [14]:
lr, epsilon, decay = (0.01, None, 0.0)
_optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 17s 1s/step - loss: 2.6224 - acc: 0.4320 - val_loss: 1.7292 - val_acc: 0.5801

Epoch 00001: val_acc improved from -inf to 0.58008, saving model to /data/oxford102/experiments/1546273533.410461/mobilenet_v2_1546273533.410461.h5
Epoch 2/100
17/17 [==============================] - 9s 532ms/step - loss: 1.3963 - acc: 0.6618 - val_loss: 1.1977 - val_acc: 0.7101

Epoch 00002: val_acc improved from 0.58008 to 0.71014, saving model to /data/oxford102/experiments/1546273533.410461/mobilenet_v2_1546273533.410461.h5
Epoch 3/100
17/17 [==============================] - 11s 675ms/step - loss: 0.4968 - acc: 0.9114 - val_loss: 0.8427 - val_acc: 0.8012

Epoch 00003: val_acc improved from 0.71014 to 0.80124, saving model to /data/oxford102/experiments/1546273533.410461/mobilenet_v2_1546273533.410461.h5
Epoch 4/100
17/17 [==============================] - 14s 818m

In [15]:
lr, epsilon, decay = (0.01, None, 0.0)
_optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 17s 1s/step - loss: 2.6914 - acc: 0.4283 - val_loss: 1.7614 - val_acc: 0.5996

Epoch 00001: val_acc improved from -inf to 0.59961, saving model to /data/oxford102/experiments/1546273756.7015753/mobilenet_v2_1546273756.7015753.h5
Epoch 2/100
17/17 [==============================] - 9s 543ms/step - loss: 1.4666 - acc: 0.6636 - val_loss: 1.2041 - val_acc: 0.7019

Epoch 00002: val_acc improved from 0.59961 to 0.70186, saving model to /data/oxford102/experiments/1546273756.7015753/mobilenet_v2_1546273756.7015753.h5
Epoch 3/100
17/17 [==============================] - 12s 700ms/step - loss: 0.5202 - acc: 0.9054 - val_loss: 0.9071 - val_acc: 0.7774

Epoch 00003: val_acc improved from 0.70186 to 0.77743, saving model to /data/oxford102/experiments/1546273756.7015753/mobilenet_v2_1546273756.7015753.h5
Epoch 4/100
17/17 [==============================] - 14

In [16]:
params['network_name'] = "inception_resnet_v2"
_optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 44s 3s/step - loss: 3.5302 - acc: 0.2385 - val_loss: 2.8001 - val_acc: 0.3574

Epoch 00001: val_acc improved from -inf to 0.35742, saving model to /data/oxford102/experiments/1546274113.3287373/inception_resnet_v2_1546274113.3287373.h5
Epoch 2/100
17/17 [==============================] - 37s 2s/step - loss: 2.5906 - acc: 0.3842 - val_loss: 2.1746 - val_acc: 0.4586

Epoch 00002: val_acc improved from 0.35742 to 0.45859, saving model to /data/oxford102/experiments/1546274113.3287373/inception_resnet_v2_1546274113.3287373.h5
Epoch 3/100
17/17 [==============================] - 38s 2s/step - loss: 1.6877 - acc: 0.5794 - val_loss: 1.8934 - val_acc: 0.5207

Epoch 00003: val_acc improved from 0.45859 to 0.52070, saving model to /data/oxford102/experiments/1546274113.3287373/inception_resnet_v2_1546274113.3287373.h5
Epoch 4/100
17/17 [====================

In [17]:
params['network_name'] = "inception_resnet_v2"
_optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 42s 2s/step - loss: 3.5376 - acc: 0.2477 - val_loss: 2.8197 - val_acc: 0.3516

Epoch 00001: val_acc improved from -inf to 0.35156, saving model to /data/oxford102/experiments/1546274823.916643/inception_resnet_v2_1546274823.916643.h5
Epoch 2/100
17/17 [==============================] - 35s 2s/step - loss: 2.5366 - acc: 0.3929 - val_loss: 2.1517 - val_acc: 0.4710

Epoch 00002: val_acc improved from 0.35156 to 0.47101, saving model to /data/oxford102/experiments/1546274823.916643/inception_resnet_v2_1546274823.916643.h5
Epoch 3/100
17/17 [==============================] - 35s 2s/step - loss: 1.6405 - acc: 0.5947 - val_loss: 1.9239 - val_acc: 0.5145

Epoch 00003: val_acc improved from 0.47101 to 0.51449, saving model to /data/oxford102/experiments/1546274823.916643/inception_resnet_v2_1546274823.916643.h5
Epoch 4/100
17/17 [==========================

In [13]:
params['network_name'] = "vgg16"
_optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 43s 3s/step - loss: 2.7948 - acc: 0.4076 - val_loss: 1.7120 - val_acc: 0.6221

Epoch 00001: val_acc improved from -inf to 0.62207, saving model to /data/oxford102/experiments/1546298852.16209/vgg16_1546298852.16209.h5
Epoch 2/100
17/17 [==============================] - 34s 2s/step - loss: 1.3107 - acc: 0.6962 - val_loss: 1.1244 - val_acc: 0.7474

Epoch 00002: val_acc improved from 0.62207 to 0.74741, saving model to /data/oxford102/experiments/1546298852.16209/vgg16_1546298852.16209.h5
Epoch 3/100
17/17 [==============================] - 37s 2s/step - loss: 0.3539 - acc: 0.9445 - val_loss: 0.8764 - val_acc: 0.7826

Epoch 00003: val_acc improved from 0.74741 to 0.78261, saving model to /data/oxford102/experiments/1546298852.16209/vgg16_1546298852.16209.h5
Epoch 4/100
17/17 [==============================] - 29s 2s/step - loss: 0.1763 - acc: 0.9821

In [14]:
params['network_name'] = "vgg16"
_optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 30s 2s/step - loss: 2.7452 - acc: 0.4154 - val_loss: 1.6841 - val_acc: 0.6230

Epoch 00001: val_acc improved from -inf to 0.62305, saving model to /data/oxford102/experiments/1546299287.931627/vgg16_1546299287.931627.h5
Epoch 2/100
17/17 [==============================] - 28s 2s/step - loss: 1.3798 - acc: 0.6746 - val_loss: 1.1194 - val_acc: 0.7329

Epoch 00002: val_acc improved from 0.62305 to 0.73292, saving model to /data/oxford102/experiments/1546299287.931627/vgg16_1546299287.931627.h5
Epoch 3/100
17/17 [==============================] - 28s 2s/step - loss: 0.3595 - acc: 0.9436 - val_loss: 0.9463 - val_acc: 0.7640

Epoch 00003: val_acc improved from 0.73292 to 0.76398, saving model to /data/oxford102/experiments/1546299287.931627/vgg16_1546299287.931627.h5
Epoch 4/100
17/17 [==============================] - 28s 2s/step - loss: 0.1838 - acc: 

In [15]:
params['network_name'] = "vgg19"
_optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 35s 2s/step - loss: 2.7727 - acc: 0.4072 - val_loss: 1.7249 - val_acc: 0.6035

Epoch 00001: val_acc improved from -inf to 0.60352, saving model to /data/oxford102/experiments/1546299847.9743216/vgg19_1546299847.9743216.h5
Epoch 2/100
17/17 [==============================] - 33s 2s/step - loss: 1.4036 - acc: 0.6682 - val_loss: 1.1962 - val_acc: 0.7081

Epoch 00002: val_acc improved from 0.60352 to 0.70807, saving model to /data/oxford102/experiments/1546299847.9743216/vgg19_1546299847.9743216.h5
Epoch 3/100
17/17 [==============================] - 33s 2s/step - loss: 0.4030 - acc: 0.9390 - val_loss: 0.9127 - val_acc: 0.7764

Epoch 00003: val_acc improved from 0.70807 to 0.77640, saving model to /data/oxford102/experiments/1546299847.9743216/vgg19_1546299847.9743216.h5
Epoch 4/100
17/17 [==============================] - 33s 2s/step - loss: 0.2074 -

In [16]:
params['network_name'] = "vgg19"
_optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 35s 2s/step - loss: 2.8277 - acc: 0.4007 - val_loss: 1.7348 - val_acc: 0.5986

Epoch 00001: val_acc improved from -inf to 0.59863, saving model to /data/oxford102/experiments/1546300275.2957659/vgg19_1546300275.2957659.h5
Epoch 2/100
17/17 [==============================] - 33s 2s/step - loss: 1.4453 - acc: 0.6664 - val_loss: 1.1902 - val_acc: 0.7288

Epoch 00002: val_acc improved from 0.59863 to 0.72878, saving model to /data/oxford102/experiments/1546300275.2957659/vgg19_1546300275.2957659.h5
Epoch 3/100
17/17 [==============================] - 33s 2s/step - loss: 0.4226 - acc: 0.9343 - val_loss: 0.9620 - val_acc: 0.7754

Epoch 00003: val_acc improved from 0.72878 to 0.77536, saving model to /data/oxford102/experiments/1546300275.2957659/vgg19_1546300275.2957659.h5
Epoch 4/100
17/17 [==============================] - 33s 2s/step - loss: 0.2207 -

# Adamax

In [16]:
params = {'network_name': "mobilenet_v2",
         'image_aug': False,
         'optimizer_name': "adam", 
         'optimizer': optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
         'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (224, 224),
         'log_time': None}

In [18]:
_optimizer = optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 18s 1s/step - loss: 2.7944 - acc: 0.3966 - val_loss: 1.8988 - val_acc: 0.5664

Epoch 00001: val_acc improved from -inf to 0.56641, saving model to /data/oxford102/experiments/1546300754.773972/mobilenet_v2_1546300754.773972.h5
Epoch 2/100
17/17 [==============================] - 10s 575ms/step - loss: 1.5841 - acc: 0.6383 - val_loss: 1.3153 - val_acc: 0.6998

Epoch 00002: val_acc improved from 0.56641 to 0.69979, saving model to /data/oxford102/experiments/1546300754.773972/mobilenet_v2_1546300754.773972.h5
Epoch 3/100
17/17 [==============================] - 13s 738ms/step - loss: 0.6337 - acc: 0.8796 - val_loss: 1.0164 - val_acc: 0.7536

Epoch 00003: val_acc improved from 0.69979 to 0.75362, saving model to /data/oxford102/experiments/1546300754.773972/mobilenet_v2_1546300754.773972.h5
Epoch 4/100
17/17 [==============================] - 14s 810

In [19]:
params['network_name'] = "inception_resnet_v2"
_optimizer = optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 48s 3s/step - loss: 3.5963 - acc: 0.2348 - val_loss: 2.9234 - val_acc: 0.3076

Epoch 00001: val_acc improved from -inf to 0.30762, saving model to /data/oxford102/experiments/1546301159.451696/inception_resnet_v2_1546301159.451696.h5
Epoch 2/100
17/17 [==============================] - 38s 2s/step - loss: 2.6572 - acc: 0.3658 - val_loss: 2.4095 - val_acc: 0.4068

Epoch 00002: val_acc improved from 0.30762 to 0.40683, saving model to /data/oxford102/experiments/1546301159.451696/inception_resnet_v2_1546301159.451696.h5
Epoch 3/100
17/17 [==============================] - 39s 2s/step - loss: 1.8428 - acc: 0.5477 - val_loss: 1.9924 - val_acc: 0.4907

Epoch 00003: val_acc improved from 0.40683 to 0.49068, saving model to /data/oxford102/experiments/1546301159.451696/inception_resnet_v2_1546301159.451696.h5
Epoch 4/100
17/17 [==========================

In [20]:
params['network_name'] = "vgg16"
_optimizer = optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 33s 2s/step - loss: 2.8074 - acc: 0.4141 - val_loss: 1.8225 - val_acc: 0.5947

Epoch 00001: val_acc improved from -inf to 0.59473, saving model to /data/oxford102/experiments/1546302091.8038983/vgg16_1546302091.8038983.h5
Epoch 2/100
17/17 [==============================] - 29s 2s/step - loss: 1.4943 - acc: 0.6604 - val_loss: 1.2412 - val_acc: 0.7019

Epoch 00002: val_acc improved from 0.59473 to 0.70186, saving model to /data/oxford102/experiments/1546302091.8038983/vgg16_1546302091.8038983.h5
Epoch 3/100
17/17 [==============================] - 28s 2s/step - loss: 0.4918 - acc: 0.9237 - val_loss: 1.0589 - val_acc: 0.7557

Epoch 00003: val_acc improved from 0.70186 to 0.75569, saving model to /data/oxford102/experiments/1546302091.8038983/vgg16_1546302091.8038983.h5
Epoch 4/100
17/17 [==============================] - 29s 2s/step - loss: 0.2926 -

In [17]:
params['network_name'] = "vgg19"
_optimizer = optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 45s 3s/step - loss: 2.7548 - acc: 0.4260 - val_loss: 1.8550 - val_acc: 0.5742

Epoch 00001: val_acc improved from -inf to 0.57422, saving model to /data/oxford102/experiments/1546302688.0139732/vgg19_1546302688.0139732.h5
Epoch 2/100
17/17 [==============================] - 39s 2s/step - loss: 1.4960 - acc: 0.6544 - val_loss: 1.2639 - val_acc: 0.7153

Epoch 00002: val_acc improved from 0.57422 to 0.71532, saving model to /data/oxford102/experiments/1546302688.0139732/vgg19_1546302688.0139732.h5
Epoch 3/100
17/17 [==============================] - 42s 2s/step - loss: 0.5176 - acc: 0.9182 - val_loss: 1.0592 - val_acc: 0.7526

Epoch 00003: val_acc improved from 0.71532 to 0.75259, saving model to /data/oxford102/experiments/1546302688.0139732/vgg19_1546302688.0139732.h5
Epoch 4/100
17/17 [==============================] - 34s 2s/step - loss: 0.3027 -

# Nadam

In [16]:
params = {'network_name': "mobilenet_v2",
         'image_aug': False,
         'optimizer_name': "adam", 
         'optimizer': optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
         'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (224, 224),
         'log_time': None}

In [19]:
_optimizer = optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 19s 1s/step - loss: 2.5060 - acc: 0.4646 - val_loss: 1.4937 - val_acc: 0.6484

Epoch 00001: val_acc improved from -inf to 0.64844, saving model to /data/oxford102/experiments/1546303120.8429782/mobilenet_v2_1546303120.8429782.h5
Epoch 2/100
17/17 [==============================] - 10s 614ms/step - loss: 1.1118 - acc: 0.7381 - val_loss: 0.9246 - val_acc: 0.7857

Epoch 00002: val_acc improved from 0.64844 to 0.78571, saving model to /data/oxford102/experiments/1546303120.8429782/mobilenet_v2_1546303120.8429782.h5
Epoch 3/100
17/17 [==============================] - 13s 756ms/step - loss: 0.3039 - acc: 0.9482 - val_loss: 0.7225 - val_acc: 0.8178

Epoch 00003: val_acc improved from 0.78571 to 0.81781, saving model to /data/oxford102/experiments/1546303120.8429782/mobilenet_v2_1546303120.8429782.h5
Epoch 4/100
17/17 [==============================] - 1

In [20]:
params['network_name'] = "inception_resnet_v2"
_optimizer = optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 47s 3s/step - loss: 3.2813 - acc: 0.2744 - val_loss: 2.4669 - val_acc: 0.4053

Epoch 00001: val_acc improved from -inf to 0.40527, saving model to /data/oxford102/experiments/1546303364.6238375/inception_resnet_v2_1546303364.6238375.h5
Epoch 2/100
17/17 [==============================] - 38s 2s/step - loss: 2.2292 - acc: 0.4531 - val_loss: 1.9273 - val_acc: 0.5259

Epoch 00002: val_acc improved from 0.40527 to 0.52588, saving model to /data/oxford102/experiments/1546303364.6238375/inception_resnet_v2_1546303364.6238375.h5
Epoch 3/100
17/17 [==============================] - 40s 2s/step - loss: 1.2984 - acc: 0.6917 - val_loss: 1.5588 - val_acc: 0.6242

Epoch 00003: val_acc improved from 0.52588 to 0.62422, saving model to /data/oxford102/experiments/1546303364.6238375/inception_resnet_v2_1546303364.6238375.h5
Epoch 4/100
17/17 [====================

In [21]:
params['network_name'] = "vgg16"
_optimizer = optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 32s 2s/step - loss: 2.4716 - acc: 0.4706 - val_loss: 1.4268 - val_acc: 0.6709

Epoch 00001: val_acc improved from -inf to 0.67090, saving model to /data/oxford102/experiments/1546304347.075828/vgg16_1546304347.075828.h5
Epoch 2/100
17/17 [==============================] - 29s 2s/step - loss: 1.0991 - acc: 0.7445 - val_loss: 0.9220 - val_acc: 0.7940

Epoch 00002: val_acc improved from 0.67090 to 0.79400, saving model to /data/oxford102/experiments/1546304347.075828/vgg16_1546304347.075828.h5
Epoch 3/100
17/17 [==============================] - 29s 2s/step - loss: 0.2359 - acc: 0.9652 - val_loss: 0.8089 - val_acc: 0.8116

Epoch 00003: val_acc improved from 0.79400 to 0.81159, saving model to /data/oxford102/experiments/1546304347.075828/vgg16_1546304347.075828.h5
Epoch 4/100
17/17 [==============================] - 28s 2s/step - loss: 0.1006 - acc: 

In [22]:
params['network_name'] = "vgg19"
_optimizer = optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 36s 2s/step - loss: 2.4515 - acc: 0.4701 - val_loss: 1.4599 - val_acc: 0.6592

Epoch 00001: val_acc improved from -inf to 0.65918, saving model to /data/oxford102/experiments/1546304767.6417031/vgg19_1546304767.6417031.h5
Epoch 2/100
17/17 [==============================] - 33s 2s/step - loss: 1.1295 - acc: 0.7335 - val_loss: 0.9493 - val_acc: 0.7712

Epoch 00002: val_acc improved from 0.65918 to 0.77122, saving model to /data/oxford102/experiments/1546304767.6417031/vgg19_1546304767.6417031.h5
Epoch 3/100
17/17 [==============================] - 33s 2s/step - loss: 0.2492 - acc: 0.9620 - val_loss: 0.7858 - val_acc: 0.8147

Epoch 00003: val_acc improved from 0.77122 to 0.81470, saving model to /data/oxford102/experiments/1546304767.6417031/vgg19_1546304767.6417031.h5
Epoch 4/100
17/17 [==============================] - 33s 2s/step - loss: 0.1143 -

In [ ]:
# SGD

In [15]:
params = {'network_name': "vgg16",
         'image_aug': False,
         'optimizer_name': "SGD", 
         'optimizer': optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False),
         'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (224, 224),
         'log_time': None}

In [11]:
params['network_name'] = "vgg16"
params['optimizer_name'] = "SGD"
_optimizer = optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 39s 2s/step - loss: 4.5988 - acc: 0.0767 - val_loss: 3.8980 - val_acc: 0.1631

Epoch 00001: val_acc improved from -inf to 0.16309, saving model to /data/oxford102/experiments/1546749875.1105452/vgg16_1546749875.1105452.h5
Epoch 2/100
17/17 [==============================] - 33s 2s/step - loss: 3.4646 - acc: 0.2408 - val_loss: 3.0891 - val_acc: 0.3188

Epoch 00002: val_acc improved from 0.16309 to 0.31884, saving model to /data/oxford102/experiments/1546749875.1105452/vgg16_1546749875.1105452.h5
Epoch 3/100
17/17 [==============================] - 37s 2s/step - loss: 2.3284 - acc: 0.4841 - val_loss: 2.7484 - val_acc: 0.3954

Epoch 00003: val_acc improved from 0.31884 to 0.39545, saving model to /data/oxford102/experiments/1546749875.1105452/vgg16_1546749875.1105452.h5
Epoch 4/100
17/17 [==============================] - 28s 2s/step - loss: 1.9892 -

In [12]:
params['network_name'] = "vgg19"
params['optimizer_name'] = "SGD"
_optimizer = optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 35s 2s/step - loss: 4.5630 - acc: 0.0836 - val_loss: 3.7900 - val_acc: 0.1797

Epoch 00001: val_acc improved from -inf to 0.17969, saving model to /data/oxford102/experiments/1546750293.552338/vgg19_1546750293.552338.h5
Epoch 2/100
17/17 [==============================] - 32s 2s/step - loss: 3.4270 - acc: 0.2413 - val_loss: 3.0336 - val_acc: 0.3261

Epoch 00002: val_acc improved from 0.17969 to 0.32609, saving model to /data/oxford102/experiments/1546750293.552338/vgg19_1546750293.552338.h5
Epoch 3/100
17/17 [==============================] - 32s 2s/step - loss: 2.3005 - acc: 0.4808 - val_loss: 2.6558 - val_acc: 0.4110

Epoch 00003: val_acc improved from 0.32609 to 0.41097, saving model to /data/oxford102/experiments/1546750293.552338/vgg19_1546750293.552338.h5
Epoch 4/100
17/17 [==============================] - 32s 2s/step - loss: 1.9894 - acc: 

In [13]:
params['network_name'] = "inception_resnet_v2"
params['optimizer_name'] = "SGD"
_optimizer = optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 44s 3s/step - loss: 4.1383 - acc: 0.1434 - val_loss: 3.2948 - val_acc: 0.2646

Epoch 00001: val_acc improved from -inf to 0.26465, saving model to /data/oxford102/experiments/1546751082.4979792/inception_resnet_v2_1546751082.4979792.h5
Epoch 2/100
17/17 [==============================] - 37s 2s/step - loss: 3.0353 - acc: 0.3125 - val_loss: 2.7439 - val_acc: 0.3696

Epoch 00002: val_acc improved from 0.26465 to 0.36957, saving model to /data/oxford102/experiments/1546751082.4979792/inception_resnet_v2_1546751082.4979792.h5
Epoch 3/100
17/17 [==============================] - 39s 2s/step - loss: 2.2819 - acc: 0.4796 - val_loss: 2.4000 - val_acc: 0.4431

Epoch 00003: val_acc improved from 0.36957 to 0.44306, saving model to /data/oxford102/experiments/1546751082.4979792/inception_resnet_v2_1546751082.4979792.h5
Epoch 4/100
17/17 [====================

In [15]:
params['network_name'] = "mobilenet_v2"
params['optimizer_name'] = "SGD"
_optimizer = optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 18s 1s/step - loss: 3.7616 - acc: 0.2151 - val_loss: 2.7583 - val_acc: 0.3945

Epoch 00001: val_acc improved from -inf to 0.39453, saving model to /data/oxford102/experiments/1546753858.6357646/mobilenet_v2_1546753858.6357646.h5
Epoch 2/100
17/17 [==============================] - 9s 515ms/step - loss: 2.3164 - acc: 0.4775 - val_loss: 1.9041 - val_acc: 0.5745

Epoch 00002: val_acc improved from 0.39453 to 0.57453, saving model to /data/oxford102/experiments/1546753858.6357646/mobilenet_v2_1546753858.6357646.h5
Epoch 3/100
17/17 [==============================] - 12s 713ms/step - loss: 1.1986 - acc: 0.7585 - val_loss: 1.6312 - val_acc: 0.6335

Epoch 00003: val_acc improved from 0.57453 to 0.63354, saving model to /data/oxford102/experiments/1546753858.6357646/mobilenet_v2_1546753858.6357646.h5
Epoch 4/100
17/17 [==============================] - 13

In [16]:
params['network_name'] = "mobilenet_v2"
params['optimizer_name'] = "SGD"
_optimizer = optimizers.SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 19s 1s/step - loss: 5.2747 - acc: 0.0133 - val_loss: 4.8617 - val_acc: 0.0381

Epoch 00001: val_acc improved from -inf to 0.03809, saving model to /data/oxford102/experiments/1546754274.016218/mobilenet_v2_1546754274.016218.h5
Epoch 2/100
17/17 [==============================] - 8s 482ms/step - loss: 4.7124 - acc: 0.0418 - val_loss: 4.4052 - val_acc: 0.0756

Epoch 00002: val_acc improved from 0.03809 to 0.07557, saving model to /data/oxford102/experiments/1546754274.016218/mobilenet_v2_1546754274.016218.h5
Epoch 3/100
17/17 [==============================] - 12s 732ms/step - loss: 3.9972 - acc: 0.1334 - val_loss: 4.0063 - val_acc: 0.1646

Epoch 00003: val_acc improved from 0.07557 to 0.16460, saving model to /data/oxford102/experiments/1546754274.016218/mobilenet_v2_1546754274.016218.h5
Epoch 4/100
17/17 [==============================] - 14s 836m

In [17]:
params['network_name'] = "mobilenet_v2"
params['optimizer_name'] = "SGD"
_optimizer = optimizers.SGD(lr=0.1, momentum=0.0, decay=0.0, nesterov=False)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 17s 1s/step - loss: 2.7860 - acc: 0.4099 - val_loss: 1.8418 - val_acc: 0.5869

Epoch 00001: val_acc improved from -inf to 0.58691, saving model to /data/oxford102/experiments/1546754934.4831538/mobilenet_v2_1546754934.4831538.h5
Epoch 2/100
17/17 [==============================] - 8s 483ms/step - loss: 1.5499 - acc: 0.6498 - val_loss: 1.3357 - val_acc: 0.7050

Epoch 00002: val_acc improved from 0.58691 to 0.70497, saving model to /data/oxford102/experiments/1546754934.4831538/mobilenet_v2_1546754934.4831538.h5
Epoch 3/100
17/17 [==============================] - 12s 734ms/step - loss: 0.6693 - acc: 0.8930 - val_loss: 1.0429 - val_acc: 0.7578

Epoch 00003: val_acc improved from 0.70497 to 0.75776, saving model to /data/oxford102/experiments/1546754934.4831538/mobilenet_v2_1546754934.4831538.h5
Epoch 4/100
17/17 [==============================] - 13

# repeat since acc very high

In [17]:
params['network_name'] = "mobilenet_v2"
params['optimizer_name'] = "SGD"
_optimizer = optimizers.SGD(lr=0.1, momentum=0.0, decay=0.0, nesterov=False)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 18s 1s/step - loss: 2.6976 - acc: 0.4177 - val_loss: 1.7442 - val_acc: 0.6045

Epoch 00001: val_acc improved from -inf to 0.60449, saving model to /data/oxford102/experiments/1546811162.66211/mobilenet_v2_1546811162.66211.h5
Epoch 2/100
17/17 [==============================] - 8s 500ms/step - loss: 1.4425 - acc: 0.6733 - val_loss: 1.2568 - val_acc: 0.7122

Epoch 00002: val_acc improved from 0.60449 to 0.71222, saving model to /data/oxford102/experiments/1546811162.66211/mobilenet_v2_1546811162.66211.h5
Epoch 3/100
17/17 [==============================] - 13s 785ms/step - loss: 0.6218 - acc: 0.8897 - val_loss: 1.0024 - val_acc: 0.7733

Epoch 00003: val_acc improved from 0.71222 to 0.77329, saving model to /data/oxford102/experiments/1546811162.66211/mobilenet_v2_1546811162.66211.h5
Epoch 4/100
17/17 [==============================] - 13s 773ms/step

In [18]:
# RMSPROP

In [19]:
params['network_name'] = "vgg16"
params['optimizer_name'] = "RMSprop"
_optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 32s 2s/step - loss: 2.4028 - acc: 0.4936 - val_loss: 1.5220 - val_acc: 0.6611

Epoch 00001: val_acc improved from -inf to 0.66113, saving model to /data/oxford102/experiments/1546755154.390664/vgg16_1546755154.390664.h5
Epoch 2/100
17/17 [==============================] - 28s 2s/step - loss: 1.1838 - acc: 0.7376 - val_loss: 1.0485 - val_acc: 0.7598

Epoch 00002: val_acc improved from 0.66113 to 0.75983, saving model to /data/oxford102/experiments/1546755154.390664/vgg16_1546755154.390664.h5
Epoch 3/100
17/17 [==============================] - 28s 2s/step - loss: 0.2884 - acc: 0.9597 - val_loss: 0.8874 - val_acc: 0.7940

Epoch 00003: val_acc improved from 0.75983 to 0.79400, saving model to /data/oxford102/experiments/1546755154.390664/vgg16_1546755154.390664.h5
Epoch 4/100
17/17 [==============================] - 29s 2s/step - loss: 0.1429 - acc: 

In [20]:
params['network_name'] = "vgg19"
params['optimizer_name'] = "RMSprop"
_optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 36s 2s/step - loss: 2.4633 - acc: 0.4802 - val_loss: 1.5107 - val_acc: 0.6729

Epoch 00001: val_acc improved from -inf to 0.67285, saving model to /data/oxford102/experiments/1546755535.920222/vgg19_1546755535.920222.h5
Epoch 2/100
17/17 [==============================] - 33s 2s/step - loss: 1.2405 - acc: 0.7096 - val_loss: 1.0181 - val_acc: 0.7754

Epoch 00002: val_acc improved from 0.67285 to 0.77536, saving model to /data/oxford102/experiments/1546755535.920222/vgg19_1546755535.920222.h5
Epoch 3/100
17/17 [==============================] - 33s 2s/step - loss: 0.2995 - acc: 0.9565 - val_loss: 0.8805 - val_acc: 0.7909

Epoch 00003: val_acc improved from 0.77536 to 0.79089, saving model to /data/oxford102/experiments/1546755535.920222/vgg19_1546755535.920222.h5
Epoch 4/100
17/17 [==============================] - 33s 2s/step - loss: 0.1765 - acc: 

In [21]:
params['network_name'] = "inception_resnet_v2"
params['optimizer_name'] = "RMSprop"
_optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 45s 3s/step - loss: 3.3076 - acc: 0.2707 - val_loss: 2.5562 - val_acc: 0.3975

Epoch 00001: val_acc improved from -inf to 0.39746, saving model to /data/oxford102/experiments/1546756006.1082816/inception_resnet_v2_1546756006.1082816.h5
Epoch 2/100
17/17 [==============================] - 36s 2s/step - loss: 2.3523 - acc: 0.4513 - val_loss: 2.0389 - val_acc: 0.5280

Epoch 00002: val_acc improved from 0.39746 to 0.52795, saving model to /data/oxford102/experiments/1546756006.1082816/inception_resnet_v2_1546756006.1082816.h5
Epoch 3/100
17/17 [==============================] - 36s 2s/step - loss: 1.5828 - acc: 0.6434 - val_loss: 1.8387 - val_acc: 0.5735

Epoch 00003: val_acc improved from 0.52795 to 0.57350, saving model to /data/oxford102/experiments/1546756006.1082816/inception_resnet_v2_1546756006.1082816.h5
Epoch 4/100
17/17 [====================

In [16]:
params['network_name'] = "mobilenet_v2"
params['optimizer_name'] = "RMSprop"
_optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], input_processor)
train_model(params, _model, (train_generator, validation_generator))

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 22s 1s/step - loss: 2.4331 - acc: 0.4752 - val_loss: 1.4218 - val_acc: 0.6689

Epoch 00001: val_acc improved from -inf to 0.66895, saving model to /data/oxford102/experiments/1546806727.064228/mobilenet_v2_1546806727.064228.h5
Epoch 2/100
17/17 [==============================] - 11s 671ms/step - loss: 1.1659 - acc: 0.7348 - val_loss: 1.0275 - val_acc: 0.7692

Epoch 00002: val_acc improved from 0.66895 to 0.76915, saving model to /data/oxford102/experiments/1546806727.064228/mobilenet_v2_1546806727.064228.h5
Epoch 3/100
17/17 [==============================] - 14s 852ms/step - loss: 0.3222 - acc: 0.9487 - val_loss: 0.8455 - val_acc: 0.8064

Epoch 00003: val_acc improved from 0.76915 to 0.80642, saving model to /data/oxford102/experiments/1546806727.064228/mobilenet_v2_1546806727.064228.h5
Epoch 4/100
17/17 [==============================] - 14s 815